## [환경설정] 전처리(2) tagger 및 tokenizer
- bareun 형태소 분석기 API 설치

In [16]:
# 바른의 리눅스 설치본 다운로드

!curl -LJks -H "uname:$(uname -a)" https://bareun.ai/api/get -o bareun-linux.deb
!ls

'ls'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [8]:
# 바른 설치

!uname -a
!dpkg -i bareun-linux.deb

'uname'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'dpkg'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [9]:
# 바른 모델이 GPU를 사용하기 위해서 다운받는 tensorflow 라이브러리

!curl -O https://storage.googleapis.com/tensorflow/libtensorflow/libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz
!tar -C /opt/bareun -xzf libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  392M    0 2112k    0     0  1631k      0  0:04:06  0:00:01  0:04:05 1641k
  3  392M    3 12.6M    0     0  5645k      0  0:01:11  0:00:02  0:01:09 5663k
  5  392M    5 23.3M    0     0  7269k      0  0:00:55  0:00:03  0:00:52 7285k
  8  392M    8 33.6M    0     0  8019k      0  0:00:50  0:00:04  0:00:46 8033k
 11  392M   11 44.2M    0     0  8552k      0  0:00:46  0:00:05  0:00:41 9160k
 13  392M   13 54.9M    0     0  8936k      0  0:00:44  0:00:06  0:00:38 10.5M
 16  392M   16 63.2M    0     0  9147k      0  0:00:43  0:00:07  0:00:36 10.5M
curl: (23) Failure writing output to destination, passed 16384 returned 2289
tar: could not chdir to '/opt/bareun'



In [10]:
%env BAREUN_ROOT="/opt/bareun"
%env LD_LIBRARY_PATH="/opt/bareun/lib"
!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" nohup /opt/bareun/bin/bareun&

env: BAREUN_ROOT="/opt/bareun"
env: LD_LIBRARY_PATH="/opt/bareun/lib"
The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


OSError: Background processes not supported.

In [ ]:
!ps -ef | grep bareun

In [ ]:
# Yeonwoo's API KEY

!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" /opt/bareun/bin/bareun -reg koba-MDIHR4Q-BQUUMOY-TOGE42A-NDESPKY

In [ ]:
!pip install -U bareunpy

In [ ]:
import sys
import bareunpy as brn
import google.protobuf.text_format as tf

from bareunpy import Tagger   # 이하 추가: 품사 tagger
from bareunpy import Tokenizer  # 형태소 tokenizer

## train 데이터 불러오기 및 클래스 인코딩

In [ ]:
# train data
# train_data_path ="~/aiffel/dktc/data/train.csv"

# train_data_path = "/content/drive/MyDrive/aiffel/dktc/data/train.csv"
train_data_path = "D:/DLThon/맞춤법교정/train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

In [ ]:
train_data.info()

### [train] 예측 대상 클래스 인코딩 (총4가지)
- '협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화'

In [ ]:
CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화']

encoder = LabelEncoder()
encoder.fit(CLASS_NAMES)

train_data['class'] = encoder.transform(train_data['class'])

corpus = train_data["conversation"]

In [ ]:
corpus.head()

## [일반 대화] 일반 대화 데이터셋 불러오기

In [ ]:
# 일반 대화 데이터셋 4000개
# conversations_casual_path = '/content/drive/MyDrive/aiffel/dktc/data/sampled_dataset.csv'  # text file
conversations_casual_path = 'D:/DLThon/맞춤법교정/sampled_dataset.csv'

# 인코딩을 명시적으로 지정하여 파일 읽기
casual_data = pd.read_csv(conversations_casual_path, encoding='utf-8-sig')  # UTF-8 BOM(Byte Order Mark) 파일 읽어 옴

In [ ]:
# 데이터 확인
print(casual_data.head(10))

In [ ]:
casual_data.info()

## [공통] - 정규표현식, 맞춤법 검사기 함수 정의
- 정규표현식 :
- 최종 맞춤법 검사기 API : py-hanspell (비교 위해 ET5 기반 모델을 함께 제시함)

### 정규표현식 적용 함수

In [4]:
# 정규표현식 적용 함수

def preprocess_sentence(sentence):
    sentence = sentence.strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    sentence = re.sub(r"[^가-힣a-zA-Z?.!,]+", " ", sentence)

    sentence = sentence.strip()
    return sentence


In [5]:
def load_conversations_from_df(corpus_df):
    conv_data = []

    for index, row in corpus_df.iterrows():  # 수정된 부분
        conversations = row['conversation']  # 수정된 부분
        conv_data.append(preprocess_sentence(conversations))  # 전처리 함수 적용
    return conv_data

In [6]:
# 정규표현식 적용 함수
# data 구성이 다르므로 별도 함수 정의

# 전처리 함수 수정 ver. casual data

def preprocess_sentence_Casual(sentence):     # 추후 사용자 지정 단어 사전 구축하기 : 'ㅠㅠ', 'ㅎㅎ', 'ㅋㅋ', '^^', '넹넹', '웅웅' 등
    sentence = sentence.strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    # 'ㅠㅠ', 'ㅎㅎ', 'ㅋㅋ'의 반복을 각각 'ㅠㅠ', 'ㅎㅎ', 'ㅋㅋ'로 통일
    sentence = re.sub(r"(ㅠ)+", "ㅠㅠ", sentence)  # 'ㅠㅠ' 이상 반복을 'ㅠㅠ'로
    sentence = re.sub(r"(ㅎ)+", "ㅎㅎ", sentence)  # 'ㅎㅎ' 이상 반복을 'ㅎㅎ'로
    sentence = re.sub(r"(ㅋ)+", "ㅋㅋ", sentence)  # 'ㅋㅋ' 이상 반복을 'ㅋㅋ'로
    # 한글, 알파벳, 문장 부호 및 'ㅠㅠ', 'ㅎㅎ', 'ㅋㅋ', '^^'는 남기고 나머지는 공백으로 대체
    sentence = re.sub(r"[^가-힣a-zA-Z?.!,ㅠㅠㅎㅎㅋㅋ^^]+", " ", sentence)

    sentence = sentence.strip()
    return sentence


### 맞춤법 교정기 py-hanspell, ET5(개인) 함수화

In [ ]:
def textLine_from_row(text_from_row) :  # 각 row 데이터 -> 하나의 리스트, 발화 단위 분리 (각 원소 = 하나의 발화)
    text_line = []
    for i in range(0,text_from_row.size) :
        text = text_from_row.iloc[i, 0].split('\n')
        text_line.append(text)
    return text_line


def py_hanspell_spell_correct(text_line) :  # 분리된 각 line의 여러 문장을 읽어 옴 (하나의 발화 단위 text)
    checked_data_1 = []
    print('py-hanspell 텍스트 교정 시작')
    start = time.time()

    for i in range(len(text_line)) :
        for k in range(len(text_line[i])):  # 해당 인덱스 내 리스트/문자열을 순회
            result = spell_checker.check(f'{text_line[i][k]}').checked  # 맞춤법 검사
            checked_data_1.append(result)  # 결과 저장

    end = time.time()
    length = end - start
    print(length, '초 걸렸습니다.')
    return checked_data_1


def BaseOnPLMmodelET5_spell_correct(text_line) :  # 분리된 각 line의 여러 문장을 읽어 옴 (하나의 발화 단위 text)
    checked_data_2 = []
    print('BaseOnPLMmodelET5 텍스트 교정 시작')
    start = time.time()

    for i in range(len(text_line)) :
        for k in range(len(text_line[i])):  # 해당 인덱스 내 리스트/문자열을 순회
            result = typos_corrector(f'{text_line[i][k]}') # 맞춤법 검사
            checked_data_2.append(result)  # 결과 저장

    end = time.time()
    length = end - start
    print(length, '초 걸렸습니다.')
    return checked_data_2


## [1-1] train 데이터셋 전처리

In [ ]:
conv_data = load_conversations_from_df(train_data)

print('전체 샘플 수:', len(conv_data))

In [ ]:
# 한국어 전처리 함수 (정규표현식) 적용 결과

print('전처리 후의 1081번째 샘플: {}'.format(conv_data[1081]))
print('전처리 후의 500번째 샘플: {}'.format(conv_data[500]))

## [1-2] 일반 대화 데이터셋 전처리

In [ ]:
# 정규표현식 적용
preprocessed_casual_data = casual_data.copy()

for i in range(len(preprocessed_casual_data)):
    preprocessed_casual_data.iloc[i, 0] = preprocess_sentence_Casual(preprocessed_casual_data.iloc[i, 0])

preprocessed_casual_data.head(10)


In [ ]:
conversations_class_Casual
sample_Casual = pd.DataFrame(conversations_class_Casual)

text = textLine_from_row(sample_Casual)

In [ ]:
py_hanspell_text = py_hanspell_spell_correct(text)

In [ ]:
py_hanspell_text

In [ ]:
df_py_casual = pd.DataFrame(py_hanspell_text)
df_py_casual

#### [생략 가능] BaseOnPLMmodelET5_spell_correct
- 실행 완료까지 오래 걸리므로 생략하시는 것을 추천 드립니다.

In [ ]:
ET5_text = BaseOnPLMmodelET5_spell_correct(text)

In [ ]:
df_ET5_casual = pd.DataFrame(ET5_text)
df_ET5_casual

# 전처리(2) - 품사 태깅 및 필터링
- 최종 POS Tagger 및  : bareun 형태소 분석 API
- 품사 필터링 : AntConc에서 키워드 리스트를 생성하기 위해, 품사를 필터링한 후 각 단어 옆에 품사 태그를 붙이는 방식으로 데이터를 변환하였음

- 남긴 품사 : 명사(NNG, NNP), 동사(VV), 형용사(VA) (조정 가능함)


## [환경 설정] 전처리(2) 품사 태깅 및 필터링

### API key 입력 후 정상 작동 확인

In [ ]:
# 아래에 "https://bareun.ai/"에서 이메일 인증 후 발급받은 API KEY("koba-...")를 입력해주세요. "로그인-내정보 확인"

API_KEY = "koba-MDIHR4Q-BQUUMOY-TOGE42A-NDESPKY" # <- 본인의 API KEY로 교체
# API_KEY = "koba-MDIHR4Q-BQUUMOY-TOGE42A-NDESPKY"
t = brn.Tagger(API_KEY, "localhost", 5656)

# sample code : debugging
res = t.tags(["안녕하세요. 정말 좋은 날씨네요."])
m = res.msg()
tf.PrintMessage(m, out=sys.stdout, as_utf8=True)

### API에서 분석 받은 객체의 정보를 확인

- length of tokens in sentences
- length of morphemes of first token in sentences
- lemma of first token in sentences
- first morph of first token in sentences
- tag of first morph of first token in sentences

In [ ]:
# 형태소 분석 실행한 객체를 json 형식으로 변환
# sample_code : get json object
jo = res.as_json()
print(jo)

In [ ]:
# 형태소 분석 실행한 객체를 튜플로 저장하거나, 명사, 동사만 추출
# sample_code : get tuple of pos tagging
pa = res.pos()
print(pa)
# another methods
ma = res.morphs()
print(ma)
na = res.nouns()
print(na)
va = res.verbs()
print(va)

## POS Tagger
- tagger와 tokenizer를 명확히 분리한 이유: 품사 태깅까지만 되어 있는 데이터가 필요한 경우가 있음 (EDA 등)

In [ ]:
# 태깅 코드
tagger = brn.Tagger(API_KEY, "localhost", 5656)  # 위의 변수명('t')과 일치시킬 경우, 생략하여도 무방함


### sample code

In [ ]:
# print results
res = tagger.tags(["안녕하세요.", "반가워요!"])
print(res.msg()) # debugging

In [ ]:
# get protobuf message.
m = res.msg()
tf.PrintMessage(m, out=sys.stdout, as_utf8=True)
print(tf.MessageToString(m, as_utf8=True))
print(f'length of sentences is {len(m.sentences)}')

In [ ]:
## output : 2
print(f'length of tokens in sentences[0] is {len(m.sentences[0].tokens)}')
print(f'length of morphemes of first token in sentences[0] is {len(m.sentences[0].tokens[0].morphemes)}')
print(f'lemma of first token in sentences[0] is {m.sentences[0].tokens[0].lemma}')
print(f'first morph of first token in sentences[0] is {m.sentences[0].tokens[0].morphemes[0]}')
print(f'tag of first morph of first token in sentences[0] is {m.sentences[0].tokens[0].morphemes[0].tag}')


### sample code : 사용자 지정 단어 사전 (생략)

In [ ]:
# custom dictionary
cust_dic = tagger.custom_dict("my")
cust_dic.copy_np_set({'내고유명사', '우리집고유명사'})
cust_dic.copy_cp_set({'코로나19'})
cust_dic.copy_cp_caret_set({'코로나^백신', '독감^백신'})
cust_dic.update()

# laod prev custom dict
cust_dict2 = tagger.custom_dict("my")
cust_dict2.load()

tagger.set_domain('my')
tagger.pos('코로나19는 언제 끝날까요?')

### 함수화 - json 객체 호출, 품사 getter, corpus 전체 태깅

In [ ]:
def tagging(tagger, sentences):
    res = tagger.tags([sentences])  # 문장 전체 태깅
    return res  # res 객체 반환


def get_json(res): # 객체를 json 형식으로 변환
    jo = res.as_json()
    return jo


def get_pos(res):  # 품사 태깅된 res instance로부터 형태소 추출
    jo = get_json(res)
    pa = res.pos()
    ma = res.morphs()
    na = res.nouns()
    va = res.verbs()
    return pa, ma, na, va, jo


def corpus_tagging(corpus, tagger):  # corpus 전체, 데이터(row) 내부의 line 단위로 품사 태깅 객체 생성
    tagged_data = []  # 태깅된 데이터를 저장할 리스트

    for i in range(len(corpus)):
        sentences = corpus[i].split('\n')  # '\n' 기준으로 문장 분리
        row_results = []  # 각 행에 대한 결과를 저장할 리스트

        for sentence in sentences:
            if sentence.strip():  # 빈 문장 제외
                res = tagging(tagger, sentence)
                pa, ma, na, va, jo = get_pos(res)  # 품사 태깅 결과
                row_results.append({
                    'sentence': sentence,
                    'pos': pa,
                    # 'morphs': ma,  # 필요시 추가하여 사용
                    # 'nouns': na,
                    # 'verbs': va,
                    # 'json': jo
                })  # 태깅 결과를 딕셔너리로 저장

        tagged_data.append(row_results)  # 각 행의 결과를 추가

    return tagged_data  # 태깅된 데이터 반환


def instance_info(res, i): # 필요시 사용
    m = res.msg()  # res에서 msg() 호출
    print(f'length of tokens in sentences[{i}] is {len(m.sentences)}')
    if m.sentences:
        print(f'length of morphemes of first token in sentences[{i}] is {len(m.sentences[0].tokens[0].morphemes)}')
        print(f'lemma of first token in sentences[{i}] is {m.sentences[0].tokens[0].lemma}')
        print(f'first morph of first token in sentences[{i}] is {m.sentences[0].tokens[0].morphemes[0].text.content}')
        print(f'tag of first morph of first token in sentences[{i}] is {m.sentences[0].tokens[0].morphemes[0].tag}')




## Filtering POS
- 유의미한 품사만 남긴(태깅된) text 데이터 추출

In [ ]:
def filter_pos(tagged_data, pos_to_keep):  # 품사 필터링 후 각 단어 옆에 품사 태그 붙여 반환함
    filtered_data = []

    for block in tagged_data:  # 여러 문장 블록을 포함하는 리스트
        for entry in block:  # 각 문장별로 필터링
            filtered_tokens = [
                f"{morph}/{pos}" for morph, pos in entry['pos'] if pos in pos_to_keep
            ]
            filtered_data.append(' '.join(filtered_tokens))

    return filtered_data

def save_to_csv(filtered_data, filename):  # 데이터 저장
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Filtered Morphs"])  # 헤더 생성

        for filtered_sentence in filtered_data:
            writer.writerow([filtered_sentence])

In [ ]:
def apply_tagging_and_filter(conversations_class, tagger, pos_to_keep):

    filtered_conversations = []  # 필터링된 데이터를 저장할 리스트

    # 1. 품사 태깅 수행
    tagged_data = corpus_tagging(conversations_class.tolist(), tagger)  # Series를 리스트로 변환

    # 2. filter_pos 함수 적용
    filtered_data = filter_pos(tagged_data, pos_to_keep)

    # 3. 결과 저장
    filtered_conversations.append(filtered_data)

    return filtered_conversations

# 필터링할 품사 선택 (예시: 명사(NNG, NNP), 동사(VV), 형용사(VA))
pos_to_keep = ['NNG', 'NNP', 'VV', 'VA']

In [ ]:
# 데이터가 'tagged_data'라는 변수에 있다고 가정
# sample code (코퍼스 태깅)
corpus00 = corpus[:3]
tagged_results = corpus_tagging(corpus00, tagger)

tagged_data = tagged_results

# 필요한 품사만 필터링
filtered_results = filter_pos(tagged_data, pos_to_keep)

# 필터링 결과를 CSV로 저장 (Filtered Morphs만 저장)
save_to_csv(filtered_results, 'filtered_pos_data_for_antconc.csv')


#### [생략 가능] 클래스 별 분리 후 적용 원할 경우  -전체 데이터 행에 대하여

In [ ]:
# 클래스 분리하여 저장

def train_class_split(train_data) :  # data를 class 별로 분리
  train_classes = [pd.DataFrame() for _ in range(4)]

  for i in range(4) :
    train_classes[i] = train_data[train_data['class'] == i]

  return train_classes

In [ ]:
train_classes = train_class_split(train_data)

In [ ]:
train_classes[0].head()  # 잘 저장되었는지 확인

In [ ]:
'''
# 결과를 저장할 리스트
conversations_all_classes = []

# train_classes의 각 클래스에 대해 반복
for i in range(len(train_classes)):
    # 각 클래스에서 conversation 열만 추출
    conversations = train_classes[i]['conversation']
    conversations_all_classes.append(conversations)

    print(conversations.head())

# conversations_all_classes 리스트에는 각 클래스별로 conversation 열이 저장됨
...

# train_classes[0]에서 conversation 열만 추출
conversations_class_0 = train_classes[0]['conversation']
conversations_class_1 = train_classes[1]['conversation']
conversations_class_2 = train_classes[2]['conversation']
conversations_class_3 = train_classes[3]['conversation']

In [ ]:
conversations_class_0.head()

In [ ]:
# 각 DataFrame을 CSV 파일로 저장
for i, df in enumerate(train_classes):
    df.to_csv(f'train_class_{i}.csv', index=False)

# 각 CSV 파일에 대한 다운로드 링크 제공 (Jupyter 환경에서 실행 시)
from IPython.display import FileLink, display

for i in range(len(train_classes)):
    display(FileLink(f'train_class_{i}.csv')) # 링크 에러시, 코랩 폴더 열면, 파일이 생성되어 있을 것임


In [ ]:
conv_tagged_0 = apply_tagging_and_filter(conversations_class_0, tagger, pos_to_keep)

In [ ]:
conv_tagged_1 = apply_tagging_and_filter(conversations_class_1, tagger, pos_to_keep)

In [ ]:
conv_tagged_2 = apply_tagging_and_filter(conversations_class_2, tagger, pos_to_keep)

In [ ]:
conv_tagged_3 = apply_tagging_and_filter(conversations_class_3, tagger, pos_to_keep)

In [ ]:
# 필터링 결과를 CSV로 저장 (Filtered Morphs만 저장)
save_to_csv(conv_tagged_0, 'filtered_pos_data_for_antconc_0.csv')
save_to_csv(conv_tagged_1, 'filtered_pos_data_for_antconc_1.csv')
save_to_csv(conv_tagged_2, 'filtered_pos_data_for_antconc_2.csv')
save_to_csv(conv_tagged_3, 'filtered_pos_data_for_antconc_3.csv')

# 전처리(3) - 형태소 분석, Tokenizer
- 최종 형태소 분석기 : bareun API

In [ ]:
# 토크나이징 코드

# API_KEY = "koba-MDIHR4Q-BQUUMOY-TOGE42A-NDESPKY" # Yeonwoo's API KEY

tokenizer = brn.Tokenizer(API_KEY, "localhost", 5656)   # tokenizer 생성


### 샘플 코드 : 정상 작동 확인

In [ ]:
# print results.
tokenized = tokenizer.tokenize_list(["안녕하세요.", "반가워요!"])

# get protobuf message.
m = tokenized.msg()
tf.PrintMessage(m, out=sys.stdout, as_utf8=True)
print(tf.MessageToString(m, as_utf8=True))
print(f'length of sentences is {len(m.sentences)}')
## output : 2
print(f'length of tokens in sentences[0] is {len(m.sentences[0].tokens)}')
print(f'length of segments of first token in sentences[0] is {len(m.sentences[0].tokens[0].segments)}')
print(f'tagged of first token in sentences[0] is {m.sentences[0].tokens[0].tagged}')
print(f'first segment of first token in sentences[0] is {m.sentences[0].tokens[0].segments[0]}')
print(f'hint of first morph of first token in sentences[0] is {m.sentences[0].tokens[0].segments[0].hint}')

## Advanced usage.
for sent in m.sentences:
    for token in sent.tokens:
        for m in token.segments:
            print(f'{m.text.content}/{m.hint}')

# get json object
jo = tokenized.as_json()
print(jo)

# get tuple of segments
ss = tokenized.segments()
print(ss)
ns = tokenized.nouns()
print(ns)
vs = tokenized.verbs()
print(vs)
# postpositions: 조사
ps = tokenized.postpositions()
print(ps)
# Adverbs, 부사
ass = tokenized.adverbs()
print(ass)
ss = tokenized.symbols()
print(ss)


### tokenizer
- DLThon 코퍼스 전체 적용 코드

In [ ]:
def tagging_and_tokenizing(tagger, tokenizer, sentences):
    # 품사 태깅
    tagged_res = tagger.tags([sentences])
    pa, ma, na, va, jo = get_pos(tagged_res)

    # 토큰화
    tokenized = tokenizer.tokenize_list([sentences])
    tokenized_msg = tokenized.msg()

    # tokenized 결과 추출
    tokens_info = []
    for token in tokenized_msg.sentences[0].tokens:
        tokens_info.append({
            'text': token.text.content,
            'segments': [seg.text.content for seg in token.segments],
            'tagged': token.tagged
        })

    return {
        'pos': pa,
        'morphs': ma,
        'nouns': na,
        'verbs': va,
        'json': jo,
        'tokens_info': tokens_info
    }

def corpus_tagging_and_tokenizing(corpus, tagger, tokenizer):
    tagged_data = []  # 태깅된 데이터를 저장할 리스트

    for i in range(len(corpus)):
        sentences = corpus[i].split('\n')  # '\n' 기준으로 문장 분리
        row_results = []  # 각 행에 대한 결과를 저장할 리스트
        for sentence in sentences:
            if sentence.strip():  # 빈 문장 제외
                result = tagging_and_tokenizing(tagger, tokenizer, sentence)
                row_results.append({
                    'sentence': sentence,
                    'pos': result['pos'],
                    'morphs': result['morphs'],
                    'nouns': result['nouns'],
                    'verbs': result['verbs'],
                    'json': result['json'],
                    'tokens_info': result['tokens_info']
                })

        tagged_data.append(row_results)  # 각 행의 결과를 추가

    return tagged_data  # 태깅된 데이터 반환



In [ ]:

# 태거 및 토크나이저 객체 생성 (가정된 API 호출)
# tagger = brn.Tagger(API_KEY, "localhost", 5656)
# tokenizer = brn.Tokenizer(API_KEY, "localhost", 5656)

# 예시 실행 (코퍼스 태깅과 토큰화)
corpus_tt = corpus[:3] # 정상 동작 확인 후 아래 코드 진행
# corpus_tt = corpus

tagged_and_tokenized_results = corpus_tagging_and_tokenizing(corpus_tt, tagger, tokenizer)

# 결과 확인
print(tagged_and_tokenized_results)
